# Fashion Recommendation Model Training 🎯

This notebook covers:
1. Collaborative Filtering Model
2. Content-Based Filtering
3. Deep Learning Model
4. Hybrid Model Integration
5. Model Evaluation

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Load Prepared Data

In [ ]:
# Load preprocessed data
X_train = np.load('../data/X_train.npy')
X_test = np.load('../data/X_test.npy')
y_train = np.load('../data/y_train.npy')
y_test = np.load('../data/y_test.npy')

print("Data loaded successfully!")

## 2. Collaborative Filtering Model

In [ ]:
class CollaborativeFilter:
    def __init__(self, n_factors=100):
        self.n_factors = n_factors
        
    def fit(self, ratings_matrix):
        # Perform SVD
        U, sigma, Vt = np.linalg.svd(ratings_matrix, full_matrices=False)
        
        # Keep only top n_factors
        self.user_features = U[:, :self.n_factors] @ np.diag(np.sqrt(sigma[:self.n_factors]))
        self.item_features = np.diag(np.sqrt(sigma[:self.n_factors])) @ Vt[:self.n_factors, :]
        
    def predict(self, user_id, item_id):
        return self.user_features[user_id] @ self.item_features[:, item_id]

# Create and train collaborative filter
cf_model = CollaborativeFilter(n_factors=50)
# Assuming we have a ratings matrix
ratings_matrix = np.random.rand(100, 1000)  # Example matrix
cf_model.fit(ratings_matrix)

## 3. Content-Based Filtering

In [ ]:
class ContentBasedFilter:
    def __init__(self):
        self.tfidf = TfidfVectorizer(stop_words='english')
        
    def fit(self, item_descriptions):
        self.feature_matrix = self.tfidf.fit_transform(item_descriptions)
        self.similarity_matrix = cosine_similarity(self.feature_matrix)
        
    def recommend(self, item_id, n_recommendations=5):
        item_similarities = self.similarity_matrix[item_id]
        similar_items = np.argsort(item_similarities)[::-1][1:n_recommendations+1]
        return similar_items

# Create and train content-based filter
cb_model = ContentBasedFilter()
# Assuming we have item descriptions
descriptions = ["Sample description"] * 1000  # Example descriptions
cb_model.fit(descriptions)

## 4. Deep Learning Model

In [ ]:
class FashionNet(nn.Module):
    def __init__(self, input_size, hidden_size=128, output_size=50):
        super(FashionNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size//2, output_size)
        )
        
    def forward(self, x):
        return self.model(x)

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)

# Create dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model
model = FashionNet(input_size=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
n_epochs = 10
losses = []

for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    losses.append(epoch_loss/len(train_loader))
    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {losses[-1]:.4f}')

## 5. Hybrid Model Integration

In [ ]:
class HybridRecommender:
    def __init__(self, cf_weight=0.4, cb_weight=0.3, dl_weight=0.3):
        self.cf_weight = cf_weight
        self.cb_weight = cb_weight
        self.dl_weight = dl_weight
        
    def recommend(self, user_id, item_id):
        # Get predictions from each model
        cf_pred = cf_model.predict(user_id, item_id)
        cb_pred = cb_model.recommend(item_id)[0]  # Get top recommendation
        dl_pred = model(torch.FloatTensor(X_test[item_id])).detach().numpy()
        
        # Combine predictions
        hybrid_pred = (
            self.cf_weight * cf_pred +
            self.cb_weight * cb_pred +
            self.dl_weight * dl_pred
        )
        
        return hybrid_pred

# Create hybrid recommender
hybrid_model = HybridRecommender()

# Example recommendation
user_id, item_id = 0, 0
prediction = hybrid_model.recommend(user_id, item_id)
print(f"Hybrid prediction for user {user_id}, item {item_id}: {prediction}")

## 6. Model Evaluation

In [ ]:
def evaluate_models():
    # Prepare test data
    test_users = np.random.randint(0, 100, size=1000)
    test_items = np.random.randint(0, 1000, size=1000)
    
    # Get predictions
    hybrid_preds = []
    for user, item in zip(test_users, test_items):
        pred = hybrid_model.recommend(user, item)
        hybrid_preds.append(pred)
    
    # Calculate metrics
    metrics = {
        'mae': np.mean(np.abs(np.array(hybrid_preds) - y_test[:1000])),
        'rmse': np.sqrt(np.mean((np.array(hybrid_preds) - y_test[:1000])**2))
    }
    
    return metrics

# Evaluate models
metrics = evaluate_models()
print("Model Evaluation Metrics:")
for metric, value in metrics.items():
    print(f"{metric.upper()}: {value:.4f}")

## 7. Save Models

In [ ]:
# Save models
import joblib

# Save collaborative filtering model
joblib.dump(cf_model, '../models/cf_model.joblib')

# Save content-based model
joblib.dump(cb_model, '../models/cb_model.joblib')

# Save deep learning model
torch.save(model.state_dict(), '../models/dl_model.pth')

# Save hybrid model
joblib.dump(hybrid_model, '../models/hybrid_model.joblib')

print("All models saved successfully!")